Importa el Geojson

In [17]:
import geojson

# Cargar el archivo GeoJSON
with open(r'C:\Users\elaut\OneDrive\Escritorio\Pasantia\arenaPrefect\1.2.archivo_video_completo_Import\la_calera-bogota_resampleado.geojson', 'r', encoding='utf-8') as f:
    data = geojson.load(f)

Obtiene la cantidad de muestras de una ruta de Geojson

In [18]:
muestras=len(data[2].geometry.coordinates[0])
muestras

36002

Guarda todas las coordenadas de la ruta en un Dataframe

In [31]:
import pandas as pd
df= pd.DataFrame(data[2].geometry.coordinates[0], columns=['Longitud', 'Latitud'])
df

,Longitud,Latitud
0,-74.023705,4.675747
1,-74.023750,4.675720
2,-74.023810,4.675640
3,-74.023850,4.675540
4,-74.023847,4.675544
...,...,...
477,-74.034590,4.689620
478,-74.034690,4.689270
479,-74.034730,4.689140
480,-74.034770,4.688980


Obtiene las coordenadas más proxima al punto de partida

In [34]:
import numpy as np

# Coordenada de Partida
ref_lat = 4.6744063
ref_long = -74.0361398

# Calcular la distancia euclidiana
df['Distancia'] = np.sqrt((df['Latitud'] - ref_lat)**2 + (df['Longitud'] - ref_long)**2)

# Encontrar la coordenada más cercana
coordenada_mas_cercana = df.loc[df['Distancia'].idxmin()]

# Verificar si la coordenada de referencia está en el DataFrame
coordenada_encontrada = df[(df['Latitud'] == ref_lat) & (df['Longitud'] == ref_long)]

if not coordenada_encontrada.empty:
    print("La coordenada de referencia se encuentra en el DataFrame.")
else:
    fila_mas_cercana = df['Distancia'].idxmin()
    print("La coordenada de referencia no se encuentra en el DataFrame.")
    print(f"La coordenada más cercana es:\n{coordenada_mas_cercana[['Latitud', 'Longitud']]}")
    print(f"Esta coordenada se encuentra en la fila: {fila_mas_cercana}")

La coordenada de referencia no se encuentra en el DataFrame.
La coordenada más cercana es:
Latitud      4.67439
Longitud   -74.03610
Name: 127, dtype: float64
Esta coordenada se encuentra en la fila: 127


Obtiene la coordenada más proximas al punto de llegada

In [35]:
import numpy as np

# Coordenada de referencia
ref_lat = 4.6685572
ref_long = -74.0443484

# Calcular la distancia euclidiana
df['Distancia'] = np.sqrt((df['Latitud'] - ref_lat)**2 + (df['Longitud'] - ref_long)**2)

# Encontrar la coordenada más cercana
coordenada_mas_cercana = df.loc[df['Distancia'].idxmin()]

# Verificar si la coordenada de referencia está en el DataFrame
coordenada_encontrada = df[(df['Latitud'] == ref_lat) & (df['Longitud'] == ref_long)]

if not coordenada_encontrada.empty:
    print("La coordenada de referencia se encuentra en el DataFrame.")
else:
    fila_mas_cercana = df['Distancia'].idxmin()
    print("La coordenada de referencia no se encuentra en el DataFrame.")
    print(f"La coordenada más cercana es:\n{coordenada_mas_cercana[['Latitud', 'Longitud']]}")
    print(f"Esta coordenada se encuentra en la fila: {fila_mas_cercana}")

La coordenada de referencia no se encuentra en el DataFrame.
La coordenada más cercana es:
Latitud      4.66855
Longitud   -74.04434
Name: 295, dtype: float64
Esta coordenada se encuentra en la fila: 295


Crea un geojson con la cantidad de muestras esperadas, a partir de un geojson con menos muestras

In [13]:
import geojson
import numpy as np
from scipy.interpolate import interp1d

def resample_geojson(ruta_geojson, muestras_esperadas):
    # Cargar el archivo GeoJSON
    with open(ruta_geojson, 'r', encoding='utf-8') as f:
        data = geojson.load(f)
    
    # Obtener las coordenadas de la ruta
    coordinates = data['features'][2]['geometry']['coordinates'][0]
    
    # Calcular el número de muestras actuales
    muestras_actuales = len(coordinates)
    
    # Crear un array de índices para las muestras actuales
    x_actual = np.linspace(0, 1, muestras_actuales)
    
    # Crear un array de índices para las muestras esperadas
    x_nuevo = np.linspace(0, 1, muestras_esperadas)
    
    # Interpolar las coordenadas
    interpolador = interp1d(x_actual, np.array(coordinates), axis=0, kind='linear')
    nuevas_coordenadas = interpolador(x_nuevo).tolist()
    
    # Actualizar las coordenadas en el GeoJSON
    data['features'][2]['geometry']['coordinates'][0] = nuevas_coordenadas
    
    # Crear la ruta para el nuevo archivo GeoJSON
    nueva_ruta_geojson = ruta_geojson.replace('.geojson', '_resampleado.geojson')
    
    # Guardar el nuevo archivo GeoJSON
    with open(nueva_ruta_geojson, 'w', encoding='utf-8') as f:
        geojson.dump(data, f, ensure_ascii=False, indent=2)
    
    print(f"Archivo GeoJSON resampleado guardado en: {nueva_ruta_geojson}")

# Ejemplo de uso
ruta_geojson = r'C:\Users\elaut\OneDrive\Escritorio\Pasantia\arenaPrefect\1.2.archivo_video_completo_Import\la_calera-bogota.geojson'
muestras_esperadas = 36002
resample_geojson(ruta_geojson, muestras_esperadas)

Archivo GeoJSON resampleado guardado en: C:\Users\elaut\OneDrive\Escritorio\Pasantia\arenaPrefect\1.2.archivo_video_completo_Import\la_calera-bogota_resampleado.geojson


Cruza detecciones con gps

In [24]:
import os
import json
import geojson

def agregar_gps_a_detecciones(path_ruta_geojson, path_jsons_detections):
    # Cargar el archivo GeoJSON
    with open(path_ruta_geojson, 'r', encoding='utf-8') as f:
        data_geojson = geojson.load(f)
    
    # Obtener las coordenadas de la ruta
    coordenadas_ruta = data_geojson['features'][2]['geometry']['coordinates'][0]
    
    # Verificar el número de muestras en el GeoJSON
    muestras_geojson = len(coordenadas_ruta)
    print(f"Número de muestras en el GeoJSON: {muestras_geojson}")
    
    # Listar y ordenar los archivos JSON en la carpeta de detecciones
    archivos_json = sorted([f for f in os.listdir(path_jsons_detections) if f.endswith('.json')],
                           key=lambda x: int(os.path.splitext(x)[0]))
    
    # Procesar cada archivo JSON
    for archivo in archivos_json:
        numero_frame = int(os.path.splitext(archivo)[0])
        
        # Verificar si el número de frame excede el número de muestras en el GeoJSON
        if numero_frame >= muestras_geojson:
            print(f"Advertencia: El frame {numero_frame} excede el número de muestras en el GeoJSON.")
            continue
        
        # Cargar el archivo JSON de detecciones
        with open(os.path.join(path_jsons_detections, archivo), 'r', encoding='utf-8') as f:
            data_detecciones = json.load(f)
        
        # Obtener las coordenadas GPS correspondientes al número de frame
        lng, lat = coordenadas_ruta[numero_frame]
        
        # Agregar las coordenadas GPS a cada detección
        for deteccion in data_detecciones:
            deteccion['lng'] = lng
            deteccion['lat'] = lat
        
        # Guardar el archivo JSON actualizado
        with open(os.path.join(path_jsons_detections, archivo), 'w', encoding='utf-8') as f:
            json.dump(data_detecciones, f, ensure_ascii=False, indent=2)
        
        print(f"Archivo {archivo} actualizado con coordenadas GPS.")

# Ejemplo de uso
path_ruta_geojson = r'C:\Users\elaut\OneDrive\Escritorio\Pasantia\arenaPrefect\1.2.archivo_video_completo_Import\la_calera-bogota_resampleado.geojson'
path_jsons_detections = r'C:\Users\elaut\OneDrive\Escritorio\Pasantia\arenaPrefect\9.jsons_tracking_Import\la_calera-bogota-track'
agregar_gps_a_detecciones(path_ruta_geojson, path_jsons_detections)

Número de muestras en el GeoJSON: 36002
Archivo 1.json actualizado con coordenadas GPS.
Archivo 2.json actualizado con coordenadas GPS.
Archivo 3.json actualizado con coordenadas GPS.
Archivo 4.json actualizado con coordenadas GPS.
Archivo 5.json actualizado con coordenadas GPS.
Archivo 6.json actualizado con coordenadas GPS.
Archivo 7.json actualizado con coordenadas GPS.
Archivo 8.json actualizado con coordenadas GPS.
Archivo 9.json actualizado con coordenadas GPS.
Archivo 10.json actualizado con coordenadas GPS.
Archivo 11.json actualizado con coordenadas GPS.
Archivo 12.json actualizado con coordenadas GPS.
Archivo 13.json actualizado con coordenadas GPS.
Archivo 14.json actualizado con coordenadas GPS.
Archivo 15.json actualizado con coordenadas GPS.
Archivo 16.json actualizado con coordenadas GPS.
Archivo 18.json actualizado con coordenadas GPS.
Archivo 20.json actualizado con coordenadas GPS.
Archivo 45.json actualizado con coordenadas GPS.
Archivo 49.json actualizado con coorde